> Thanks to [seefun](https://www.kaggle.com/seefun) and his [TorchUtils](https://github.com/seefun/TorchUtils)

At first, I tried only first three folds and thus create the dataset (`../input/three-pretrained-resnestpth`)

Later on, I further trained more epochs for the first three folds, and considered the last two folds. (`but it was already ddl` )

My final result (with only three folds, which is obviously improvable) is 0.98272/19 for public LB and 0.98477/19 for private LB. 

In [ ]:
import sys
sys.path.append('../input/torchutils-l/TorchUtils-master/torch_utils')
!pip install -r ../input/torchutils-l/TorchUtils-master/requirements.txt
!pip install ../input/torchutils-l/TorchUtils-master/
import os
import cv2
import time
import math
import torch
import torch.nn as nn
import numpy as np
import pandas as pd 
from sklearn.model_selection import train_test_split, StratifiedKFold
from torch.optim import Adam, AdamW
from torch.nn.parameter import Parameter
from torch.autograd import Variable
from torch.utils.data import DataLoader, Dataset
from sklearn import metrics
import urllib
import pickle
import torch.nn.functional as F
import seaborn as sns
import random
import sys
import gc
import shutil
from tqdm.autonotebook import tqdm
import albumentations
from albumentations import pytorch as AT

import scipy.special
sigmoid = lambda x: scipy.special.expit(x)
from scipy.special import softmax

import torch_utils as tu 

import warnings
warnings.filterwarnings("ignore")

In [ ]:
sys.path.append('../input/three-pretrained-resnestpth')

In [ ]:
SEED = 42
base_dir = '../input/d2lclassifyleaves/'
tu.tools.seed_everything(SEED, deterministic=False)
tu.tools.set_gpus('3') # gpu ids
EXP = 1
while os.path.exists('./exp/exp%d'%EXP):
    EXP+=1
os.makedirs('./exp/exp%d'%EXP)
CLASSES = 176
FOLD = 5
BATCH_SIZE = 64
ACCUMULATE = 1
LR = 3e-4
EPOCH = 35
DECAY_SCALE = 20.0
MIXUP = 0.2 # 0 to 1

In [ ]:
train_transform = albumentations.Compose([
    albumentations.RandomRotate90(p=0.5),
    albumentations.Transpose(p=0.5),
    albumentations.Flip(p=0.5),
    albumentations.HorizontalFlip(p=0.5),
    albumentations.ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.0625, rotate_limit=45, border_mode=1, p=0.5),
    tu.randAugment(),
    albumentations.Normalize(),
    AT.ToTensorV2(),
    ])
    
test_transform = albumentations.Compose([
    albumentations.Normalize(),
    AT.ToTensorV2(),
    ])


class LeavesDataset(Dataset):
    
    def __init__(self, df, label_encoder, data_path='../input/d2lclassifyleaves/images', transform = train_transform): 
        self.df = df 
        self.data_path = data_path
        self.transform = transform
        self.df.label = self.df.label.apply(lambda x: label_encoder[x])

    def __len__(self):
        return self.df.shape[0]

    def __getitem__(self, idx):
        img_path, label = self.df.image[idx], self.df.label[idx]
        img_path = os.path.join(self.data_path, img_path)
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = self.transform(image = img)['image']
        return img, label

In [ ]:
train_df = pd.read_csv(os.path.join(base_dir, 'train.csv'))
train_df.head()

In [ ]:
sfolder = StratifiedKFold(n_splits=FOLD,random_state=SEED,shuffle=True)
tr_folds = []
val_folds = []
for train_idx, val_idx in sfolder.split(train_df.image, train_df.label):
    tr_folds.append(train_idx)
    val_folds.append(val_idx)
    print(len(train_idx), len(val_idx))

In [ ]:
from torch.optim.lr_scheduler import CosineAnnealingLR
scaler = torch.cuda.amp.GradScaler() # for AMP training

In [ ]:
def train_model(epoch, verbose=False):
    model_conv.train()         
    avg_loss = 0.
    optimizer.zero_grad()
    if verbose:
        bar = tqdm(total=len(train_loader))
    mixup_fn = tu.Mixup(prob=MIXUP, switch_prob=0.0, onehot=True, label_smoothing=0.05, num_classes=CLASSES)
    for idx, (imgs, labels) in enumerate(train_loader):
        imgs_train, labels_train = imgs.float().cuda(), labels.cuda()
        if MIXUP:
            imgs_train, labels_train = mixup_fn(imgs_train, labels_train)
        with torch.cuda.amp.autocast():
            output_train, _ = model_conv(imgs_train)
            loss = criterion(output_train, labels_train)
        scaler.scale(loss).backward()
        if ((idx+1)%ACCUMULATE==0): # Gradient Accumulate
            scaler.step(optimizer)
            scaler.update()
            optimizer.zero_grad()
            scheduler.step()
        avg_loss += loss.item() / len(train_loader) 
        if verbose:
            bar.update(1)
    if verbose:
        bar.close()
    return avg_loss

def test_model():    
    avg_val_loss = 0.
    model_conv.eval()
    y_true_val = np.zeros(len(valset))
    y_pred_val = np.zeros((len(valset), CLASSES))
    with torch.no_grad():
        for idx, (imgs, labels) in enumerate(val_loader):
            imgs_vaild, labels_vaild = imgs.float().cuda(), labels.cuda()
            output_test, _ = model_conv(imgs_vaild)
            avg_val_loss += (criterion_test(output_test, labels_vaild).item() / len(val_loader)) 
            a = labels_vaild.detach().cpu().numpy().astype(np.int)
            b = softmax(output_test.detach().cpu().numpy(), axis=1)

            y_true_val[idx*BATCH_SIZE:idx*BATCH_SIZE+b.shape[0]] = a
            y_pred_val[idx*BATCH_SIZE:idx*BATCH_SIZE+b.shape[0]] = b
            
    metric_val = sum(np.argmax(y_pred_val, axis=1) == y_true_val) / len(y_true_val)
    return avg_val_loss, metric_val

In [ ]:
def train(fold):
    best_avg_loss = 100.0
    best_acc = 0.0

    avg_val_loss, avg_val_acc = test_model()
    print('pretrain val loss %.4f precision %.4f'%(avg_val_loss, avg_val_acc))       

    ### training
    for epoch in range(EPOCH):   
        print('lr:', optimizer.param_groups[0]['lr']) 
        np.random.seed(SEED+EPOCH*999)
        start_time = time.time()
        avg_loss = train_model(epoch)
        avg_val_loss, avg_val_acc = test_model()
        elapsed_time = time.time() - start_time 
        print('Epoch {}/{} \t train_loss={:.4f} \t val_loss={:.4f} \t val_precision={:.4f} \t time={:.2f}s'.format(
            epoch + 1, EPOCH, avg_loss, avg_val_loss, avg_val_acc, elapsed_time))

        if avg_val_loss < best_avg_loss:
            best_avg_loss = avg_val_loss

        if avg_val_acc > best_acc:
            best_acc = avg_val_acc
            torch.save(model_conv.module.state_dict(), './exp/exp' + str(EXP) + '/model-best' + str(fold) + '.pth')
            print('model saved!')

        print('=================================')   

    print('best loss:', best_avg_loss)
    print('best precision:', best_acc)
    return best_avg_loss, best_acc

### Prediction

In [ ]:
test_df = pd.read_csv(os.path.join(base_dir, 'test.csv'))
test_df.head()

In [ ]:
class LeavesDataset_test(Dataset):
    
    def __init__(self, df, data_path='../input/d2lclassifyleaves/', transform = test_transform): 
        self.df = df 
        self.data_path = data_path
        self.transform = transform
#         self.df.label = self.df.label.apply(lambda x: label_encoder[x])

    def __len__(self):
        return self.df.shape[0]

    def __getitem__(self, idx):
        img_path = self.df.image[idx]
        img_path = os.path.join(self.data_path, img_path)
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = self.transform(image = img)['image']
        return img #, label

In [ ]:
test_set = LeavesDataset_test(test_df)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=BATCH_SIZE, shuffle=False, num_workers=8)

In [ ]:
####################### Model ########################
model_conv = tu.ImageModel(name='resnest50d', pretrained=False, feature=2048, classes=CLASSES)
model_conv.eval()
model_conv.load_state_dict(torch.load('../input/three-pretrained-resnestpth/model-best0.pth'))
model_conv.cuda()
model_conv = torch.nn.DataParallel(model_conv)

model_conv1 = tu.ImageModel(name='resnest50d', pretrained=False, feature=2048, classes=CLASSES)
model_conv1.eval()
model_conv1.load_state_dict(torch.load('../input/three-pretrained-resnestpth/model-best1.pth'))
model_conv1.cuda()
model_conv1 = torch.nn.DataParallel(model_conv1)

model_conv2 = tu.ImageModel(name='resnest50d', pretrained=False, feature=2048, classes=CLASSES)
model_conv2.eval()
model_conv2.load_state_dict(torch.load('../input/three-pretrained-resnestpth/model-best2.pth'))
model_conv2.cuda()
model_conv2 = torch.nn.DataParallel(model_conv2)

model_conv3 = tu.ImageModel(name='resnest50d', pretrained=False, feature=2048, classes=CLASSES)
model_conv3.eval()
model_conv3.load_state_dict(torch.load('../input/three-pretrained-resnestpth/model-best3.pth'))
model_conv3.cuda()
model_conv3 = torch.nn.DataParallel(model_conv3)

model_conv4 = tu.ImageModel(name='resnest50d', pretrained=False, feature=2048, classes=CLASSES)
model_conv4.eval()
model_conv4.load_state_dict(torch.load('../input/three-pretrained-resnestpth/model-best4.pth'))
model_conv4.cuda()
model_conv4 = torch.nn.DataParallel(model_conv4)

In [ ]:
labels = train_df.label.unique()
label_encoder = {}
for idx, name in enumerate(labels):
    label_encoder.update({name:idx})

In [ ]:
#用数字表示对应类
class_to_num = label_encoder
#将数字转换回对应的类
num_to_class = {v: k for k, v in class_to_num.items()}

In [ ]:
# Initialize a list to store the predictions.
predictions = []
# Iterate the testing set by batches.
for batch in tqdm(test_loader):    
    imgs = batch
    with torch.no_grad():
        logits, _ = model_conv(imgs.cuda())
        logits1, _ = model_conv1(imgs.cuda())
        logits2, _ = model_conv2(imgs.cuda())
        logits3, _ = model_conv3(imgs.cuda())
        logits4, _ = model_conv4(imgs.cuda())
        mean_logits = (logits + logits1 + logits2 + logits3 + logits4)/5
    
    # Take the class with greatest logit as prediction and record it.
    predictions.extend(mean_logits.argmax(dim=-1).cpu().numpy().tolist())

    
preds = []
for i in predictions:
    preds.append(num_to_class[i])

test_data = test_df
test_data['label'] = pd.Series(preds)
submission = pd.concat([test_data['image'], test_data['label']], axis=1)
saveFileName = './submission.csv'
submission.to_csv(saveFileName, index=False)
print("Done!!!!!!!!!!!!!!!!!!!!!!!!!!!")